In [26]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict
import requests
import math
from pathlib import Path

## Setup

In [27]:
data_folder = Path('../data/state')

In [28]:
with open(data_folder/"usStates.json", 'r') as f:
    state_json = json.load(f)

In [29]:
state_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': '01',
   'properties': {'name': 'Alabama',
    'population': 4903185,
    'beds': 19000,
    'density': 94.65},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-87.359296, 35.00118],
      [-85.606675, 34.984749],
      [-85.431413, 34.124869],
      [-85.184951, 32.859696],
      [-85.069935, 32.580372],
      [-84.960397, 32.421541],
      [-85.004212, 32.322956],
      [-84.889196, 32.262709],
      [-85.058981, 32.13674],
      [-85.053504, 32.01077],
      [-85.141136, 31.840985],
      [-85.042551, 31.539753],
      [-85.113751, 31.27686],
      [-85.004212, 31.003013],
      [-85.497137, 30.997536],
      [-87.600282, 30.997536],
      [-87.633143, 30.86609],
      [-87.408589, 30.674397],
      [-87.446927, 30.510088],
      [-87.37025, 30.427934],
      [-87.518128, 30.280057],
      [-87.655051, 30.247195],
      [-87.90699, 30.411504],
      [-87.934375, 30.657966],
      [-88.011052, 30.685351

## Add state abbreviations

In [30]:
state_to_abbr = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

## Add centroids

In [31]:
with open(data_folder/"dataCovidState.json", 'r') as f:
    centroids_json = json.load(f)

In [32]:
for state in state_json["features"]:
    state_id = state["id"]
    state["properties"]["lat"] = centroids_json[state_id]["LAT"]
    state["properties"]["long"] = centroids_json[state_id]["LONG"]
    state["properties"]["abbr"] = state_to_abbr[state["properties"]["name"]]

In [33]:
state_json["features"]

[{'type': 'Feature',
  'id': '01',
  'properties': {'name': 'Alabama',
   'population': 4903185,
   'beds': 19000,
   'density': 94.65,
   'lat': 32.318231,
   'long': -86.902298,
   'abbr': 'AL'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-87.359296, 35.00118],
     [-85.606675, 34.984749],
     [-85.431413, 34.124869],
     [-85.184951, 32.859696],
     [-85.069935, 32.580372],
     [-84.960397, 32.421541],
     [-85.004212, 32.322956],
     [-84.889196, 32.262709],
     [-85.058981, 32.13674],
     [-85.053504, 32.01077],
     [-85.141136, 31.840985],
     [-85.042551, 31.539753],
     [-85.113751, 31.27686],
     [-85.004212, 31.003013],
     [-85.497137, 30.997536],
     [-87.600282, 30.997536],
     [-87.633143, 30.86609],
     [-87.408589, 30.674397],
     [-87.446927, 30.510088],
     [-87.37025, 30.427934],
     [-87.518128, 30.280057],
     [-87.655051, 30.247195],
     [-87.90699, 30.411504],
     [-87.934375, 30.657966],
     [-88.011052, 30.685351],
     [-88.

## Get and add covid data

In [34]:
#https://covidtracking.com/api/states/info <- this api has info about where the data comes from
api_url = 'https://covidtracking.com/api/states'
state_covid = requests.get(api_url).json()

In [39]:
state_covid

[{'state': 'AK',
  'positive': 1,
  'negative': 143,
  'pending': None,
  'death': None,
  'total': 144,
  'lastUpdateEt': '3/13 16:30',
  'checkTimeEt': '3/16 12:39'},
 {'state': 'AL',
  'positive': 22,
  'negative': 28,
  'pending': 46,
  'death': 0,
  'total': 96,
  'lastUpdateEt': '3/15 19:20',
  'checkTimeEt': '3/16 12:40'},
 {'state': 'AR',
  'positive': 22,
  'negative': 132,
  'pending': 14,
  'death': None,
  'total': 168,
  'lastUpdateEt': '3/16 00:00',
  'checkTimeEt': '3/16 12:41'},
 {'state': 'AZ',
  'positive': 18,
  'negative': 121,
  'pending': 50,
  'death': 0,
  'total': 189,
  'lastUpdateEt': '3/16 12:00',
  'checkTimeEt': '3/16 12:22'},
 {'state': 'CA',
  'positive': 335,
  'negative': 7981,
  'pending': None,
  'death': 6,
  'total': 8316,
  'lastUpdateEt': '3/16 07:47',
  'checkTimeEt': '3/16 12:46'},
 {'state': 'CO',
  'positive': 131,
  'negative': 627,
  'pending': None,
  'death': 1,
  'total': 758,
  'lastUpdateEt': '3/15 15:45',
  'checkTimeEt': '3/16 12:46'

In [41]:
for state1 in state_covid:
    if(state1 == {}): continue
    for state2 in state_json["features"]:
        if(state1["state"] == state2["properties"]["abbr"]):
            state2["properties"]["testing"] = {}
            state2["properties"]["deaths"] = state1["death"]
            state2["properties"]["cases"] = state1["positive"]
            state2["properties"]["lastUpdateEt"] = state1["lastUpdateEt"]
            state2["properties"]["checkTimeEt"] = state1["checkTimeEt"]
            testing_keys = ["positive", "negative", "pending", "total"]
            for key in testing_keys:
                state2["properties"]["testing"][key] = state1[key]
            

In [42]:
state_json["features"]

[{'type': 'Feature',
  'id': '01',
  'properties': {'name': 'Alabama',
   'population': 4903185,
   'beds': 19000,
   'density': 94.65,
   'lat': 32.318231,
   'long': -86.902298,
   'abbr': 'AL',
   'testing': {'positive': 22, 'negative': 28, 'pending': 46, 'total': 96},
   'deaths': 0,
   'cases': 22,
   'lastUpdateEt': '3/15 19:20',
   'checkTimeEt': '3/16 12:40'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-87.359296, 35.00118],
     [-85.606675, 34.984749],
     [-85.431413, 34.124869],
     [-85.184951, 32.859696],
     [-85.069935, 32.580372],
     [-84.960397, 32.421541],
     [-85.004212, 32.322956],
     [-84.889196, 32.262709],
     [-85.058981, 32.13674],
     [-85.053504, 32.01077],
     [-85.141136, 31.840985],
     [-85.042551, 31.539753],
     [-85.113751, 31.27686],
     [-85.004212, 31.003013],
     [-85.497137, 30.997536],
     [-87.600282, 30.997536],
     [-87.633143, 30.86609],
     [-87.408589, 30.674397],
     [-87.446927, 30.510088],
     [-87.37025

## Risks

### Local Risk

In [43]:
#111 is to convert degrees to kilometers
def get_distance(c0, c1):
    lat_dist = abs(c0[0])-abs(c1[0])
    lng_dist = abs(c0[1])-abs(c1[1])
    distance = 111 * math.sqrt(lat_dist**2 + lng_dist**2)
    return distance

In [44]:
def calc_state_local_risk(props):
    cases = props["cases"]
    population = props.get("population", -1)
    if cases == 'NaN': cases = 0
    return cases/population if population != -1 else -1

In [45]:
for state in state_json["features"]:
    if not state["properties"]["name"] == "Puerto Rico":
        state["properties"]["risk"] = {}
        state["properties"]["risk"]["LOCALRISK"] = calc_state_local_risk(state["properties"])

In [46]:
def get_state_all_neighbor_risk(props):
    risk_details = {}
    for state in state_json["features"]:
        if not state["properties"]["name"] == "Puerto Rico":
            neighbor_props = state["properties"]
            neighbor_risk = get_state_neighbor_risk(props, neighbor_props)
            if(neighbor_risk != {}): 
                risk_details[neighbor_props["abbr"]] = neighbor_risk
    props["risk"]["NEIGHBORRISK"] = calc_county_neighbor_risk(risk_details)
    props["risk"]["TOTALRISK"] = props["risk"]["NEIGHBORRISK"] + props["risk"]["LOCALRISK"]

In [47]:
def calc_county_neighbor_risk(risks):
    num_risks = len(risks)
    avg_risk = 0
    for risk in risks.values():
        distance, neighbor_risk = risk.values()
        avg_risk += (float(neighbor_risk)/num_risks)/(math.log2(float(distance)))
    return avg_risk

In [48]:
def get_state_neighbor_risk(props, neighbor_props):
    neighbor_centroid = [neighbor_props["lat"], neighbor_props["long"]]
    neighbor_risk = neighbor_props["risk"]["LOCALRISK"]
    centroid = [props["lat"], props["long"]]
    distance = get_distance(centroid, neighbor_centroid)
    if(props["abbr"] == neighbor_props["abbr"] or neighbor_risk < 0):
        return {}
    else:
        return {"DISTANCE":distance, "LOCALRISK":neighbor_risk}

In [49]:
for state in state_json["features"]:
    if not state["properties"]["name"] == "Puerto Rico":
        get_state_all_neighbor_risk(state["properties"])


In [50]:
for state in state_json["features"]:
    print(state["properties"])

{'name': 'Alabama', 'population': 4903185, 'beds': 19000, 'density': 94.65, 'lat': 32.318231, 'long': -86.902298, 'abbr': 'AL', 'testing': {'positive': 22, 'negative': 28, 'pending': 46, 'total': 96}, 'deaths': 0, 'cases': 22, 'lastUpdateEt': '3/15 19:20', 'checkTimeEt': '3/16 12:40', 'risk': {'LOCALRISK': 4.486879446726974e-06, 'NEIGHBORRISK': 9.586044693648936e-07, 'TOTALRISK': 5.445483916091868e-06}}
{'name': 'Alaska', 'population': 731545, 'beds': 1813, 'density': 1.264, 'lat': 63.588753, 'long': -154.493062, 'abbr': 'AK', 'testing': {'positive': 1, 'negative': 143, 'pending': None, 'total': 144}, 'deaths': None, 'cases': 1, 'lastUpdateEt': '3/13 16:30', 'checkTimeEt': '3/16 12:39', 'risk': {'LOCALRISK': 1.366969906157516e-06, 'NEIGHBORRISK': 8.180499778594354e-07, 'TOTALRISK': 2.1850198840169516e-06}}
{'name': 'Arizona', 'population': 7278717, 'beds': 15452, 'density': 57.05, 'lat': 34.048928, 'long': -111.093731, 'abbr': 'AZ', 'testing': {'positive': 18, 'negative': 121, 'pending

In [51]:
print(state_json["features"][0]["properties"].keys())

dict_keys(['name', 'population', 'beds', 'density', 'lat', 'long', 'abbr', 'testing', 'deaths', 'cases', 'lastUpdateEt', 'checkTimeEt', 'risk'])


## Export

In [52]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

with open("states.json", 'w') as f:
    json.dump(state_json, f, cls=NpEncoder)
